In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline 

import cv2

import os
import tensorflow
tensorflow.__version__

import os, shutil # sorting files

import matplotlib.pyplot as plt # data visualization

import torch
import kornia
import cv2
from keras.preprocessing.image import array_to_img
from keras.preprocessing.image import save_img

In [ ]:
df = pd.read_csv('../input/happy-whale-and-dolphin/train.csv')
species = df['species'].tolist()
ids = df['individual_id'].tolist()
imgList = df['image'].tolist()
os.makedirs('../working/train_species_list', exist_ok=False)

path = '../input/whale2-cropped-dataset/cropped_train_images/cropped_train_images/'
print(len(species))
for x in range(round(len(species))):
    
    # read the image with OpenCV
    img = cv2.imread(path + str(imgList[x]))
    #flip the image horizontally and make a copy
    flipped = cv2.flip(img, 1)
    # convert to torch tensor
    data = kornia.image_to_tensor(img, keepdim=False)
    data = data.float() / 255.
    # create the operator
    sharpen = kornia.filters.UnsharpMask((9,9), (2.5,2.5))
    sharpened_tensor = sharpen(data)
    # Convert back to image
    sharpened_image = kornia.utils.tensor_to_image(sharpened_tensor) 
    sharpened_image = array_to_img(sharpened_image)
    flipped = array_to_img(flipped)
        
    path3 = '../working/train_species_list/' + ids[x]
    if not (os.path.isdir(path3)):
        os.makedirs(path3, exist_ok=False)
    shutil.copy(path + str(imgList[x]), path3)
    save_img(path3 + "/" + imgList[x] + "_flipped.jpg", flipped)
    save_img(path3 + "/" + imgList[x] + "_sharpened.jpg", sharpened_image)
    if(x == 100):
        print(x)
    elif(x%1000 == 0):
        print(x)

In [ ]:
# Fixed for Cats & Dogs color images
CHANNELS = 3

IMAGE_RESIZE = 256
RESNET50_POOLING_AVERAGE = 'avg'
DENSE_LAYER_ACTIVATION = 'softmax'

OBJECTIVE_FUNCTION = 'categorical_crossentropy'

# Common accuracy metric for all outputs, but can use different metrics for different output
LOSS_METRICS = ['accuracy']

# These steps value should be proper FACTOR of no.-of-images in train & valid folders respectively
# Training images processed in each step would be no.-of-train-images / STEPS_PER_EPOCH_TRAINING
STEPS_PER_EPOCH_TRAINING = 100

# These steps value should be proper FACTOR of no.-of-images in train & valid folders respectively
# NOTE that these BATCH* are for Keras ImageDataGenerator batching to fill epoch step input
BATCH_SIZE_TRAINING = 100

# Using 1 to easily manage mapping between test_generator & prediction for submission preparation
BATCH_SIZE_TESTING = 1

In [ ]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense

### 
### Below systax is available with TensorFlow 1.11 onwards but this upgrade is not available for Kaggle kernel yet
###
#import tensorflow as tf
#print(tf.__version__)
#import tensorflow as tf
#from tf.keras.applications import ResNet50
#from tf.keras.models import Sequential

In [ ]:
#Still not talking about our train/test data or any pre-processing.

model = Sequential()

# 1st layer as the lumpsum weights from resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5
# NOTE that this layer will be set below as NOT TRAINABLE, i.e., use it as is
model.add(tensorflow.keras.applications.ResNet152(include_top = False, pooling = RESNET50_POOLING_AVERAGE, weights = 'imagenet'))

# 2nd layer as Dense for 2-class classification, i.e., dog or cat using SoftMax activation
model.add(Dense(8192, activation = 'relu'))
model.add(Dense(15587, activation = DENSE_LAYER_ACTIVATION))

# Say not to train first layer (ResNet) model as it is already trained
model.layers[0].trainable = False

In [ ]:
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

image_size = IMAGE_RESIZE

# preprocessing_function is applied on each image but only after re-sizing & augmentation (resize => augment => pre-process)
# Each of the keras.application.resnet* preprocess_input MOSTLY mean BATCH NORMALIZATION (applied on each batch) stabilize the inputs to nonlinear activation functions
# Batch Normalization helps in faster convergence
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)

# flow_From_directory generates batches of augmented data (where augmentation can be color conversion, etc)
# Both train & valid folders must have NUM_CLASSES sub-folders
train_generator = data_generator.flow_from_directory(
        '../working/train_species_list',
        target_size=(image_size, image_size),
        batch_size=BATCH_SIZE_TRAINING,
        class_mode='categorical',
        shuffle = True)

valid_generator = data_generator.flow_from_directory(
        '../input/happywhalesortedbyindividual/train_species_list',
        target_size=(image_size, image_size),
        class_mode='categorical',
        batch_size=100,
        shuffle = True)

print(train_generator.samples)

In [ ]:
# Max number of steps that these generator will have opportunity to process their source content
# len(train_generator) should be 'no. of available train images / BATCH_SIZE_TRAINING'
# len(valid_generator) should be 'no. of available train images / BATCH_SIZE_VALIDATION'
(BATCH_SIZE_TRAINING, len(train_generator),)

In [ ]:

loss = tensorflow.keras.losses.CategoricalCrossentropy()

model.compile(optimizer='sgd',
              loss=loss,
              metrics=['accuracy'])

In [ ]:
fit_history = model.fit_generator(
        train_generator,
        steps_per_epoch=len(train_generator)//BATCH_SIZE_TRAINING,
        epochs = 128,
        shuffle = True,
        validation_data=valid_generator,
)

In [ ]:
print(fit_history.history.keys())

In [ ]:
model.save_model('happy-whale-resnet-152.h5')